#   Graph implementation

In this notebook I have implemented my own classes to represent a graph and its components and other functions necessery for processing a text.<br>
A graph consists of nodes that may or may not be connected via edges. There are two easy ways to implement it in python using an adjacency list or an adjacency matrix. In this case, I have used an adjacency list because many operations are less expensive in term of complexity on this type data structure. Yet, I have added function which can transform adjacency list to an adjacency matrix.

## Important points in the implementation: 

### Complexity of building the graph

- Assuming that we have a dataset containing N samples, building a graph with this dataset can be done in 2 steps:
    1. Creating a node for each sample by iterating through the dataset which will take exactly ***O(N)*** time.
    2. Adding edges between the nodes of the graph assuming that two nodes are connected if they share at least n tokens. Since the graph is undirected, this will take only ***O(N.log(N))*** time ignoring the time we spend computing the number of shared tokens. 
    
### complexity of finding all connected components
  
- To find the component of a given graph there is many ways to do it. Here, I have implemented as follow:
    1.  I start by initializing all the nodes as not visited. Then I iterate through all nodes, and each time I chech if the node is yet visited or not. If it wasn't visited, I call the DFS function.
    2. DFS is an implementation of the Depth-First Search algorithm. In each iteration of the for loop, it adds this node to the current component and explores their neighbors in a recurring manner.
    
    
This Algorithm take also ***O(N + $\epsilon$)*** time

In [1]:
from random import choices
import pickle
import pandas as pd
import re
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords
import string

In [2]:
class Graph(object):
    """
    A class used to represent a graph
    
    Attributes
    ----------
    nodes : dictionary
        used to store the nodes of the graph
    nodesNumber : int
        number of nodes in the graph

    Methods
    -------
    add_node(node)
        adds new node to the graph
    
    add_edge(node1, node2)
        adds edges between node1 and node1  
    
    add_edges(n)
        adds the edges between each two nodes
        if they share at least n tokens
    
    get_nodes()
        returns the list of nodes in the graph
    
    samples(n=10)
        print n random nodes with their edges
    """
    
    def __init__(self):
        """ initializes a graph object
        as an empty dictionary
        """
        self.nodes = {}
        self.nodesNumber = 0
        self.connectedComponents = None
        
    def add_node(self, node):
        """ Adds a new node to the graph
        if doesn't exist already
        
        Parameters
        ----------
        node : node
            the object node to add
        """
        if node in self.nodes:
            print("node ", node, " already exists in the graph")
        else:
            self.nodes[node.id] = node
            self.nodesNumber +=1
            
    def add_edge(self, node1, node2, weight):
        """ Adds an edge between node1 and node2
        if both exist in the graph
        
        Parameters
        ----------
        node1: node
        node2: node
        """
        if node1 not in self.nodes:
            print("node ", node1.id, " doesn't exist in the graph")
        
        elif node2 not in self.nodes:
            print("node ", node2.id, " doesn't exist in the graph")
        
        else:
            self.nodes[node1].edges[node2] = weight
            self.nodes[node2].edges[node1] = weight
    
    
    def add_edges(self, n):
        """Adds adges bewteen each two nodes of the graph
        if they share at least n tokens
        
        Parameters
        ----------
        n : int
            Number of tokens that each two nodes
            should share two have a link between them
        """
        nodes = list(self.nodes.keys())
        for i in range(len(nodes)):
            for j in range(i, len(nodes)):
                number_shared_tokens = intersection(self.nodes[i], self.nodes[j])
                if (i!=j) and (number_shared_tokens >= n):
                    self.add_edge(i, j, number_shared_tokens)
    
    def get_nodes(self):
        """ returns the list of the 
            nodes exist in the graph
        """
        return list(self.nodes.keys())
    
    def samples(self, n=10):
        """Prints n random nodes form the graph.
        If the argument n isn't passed in, the 
        default value
        
        Parameters
        ----------
        n : int, optional
            Number of samples to print (default is 10)
        """
        graph = "{"
        samples = choices(list(self.nodes.keys()), k = n)
        for i in samples:
            graph += str(self.nodes[i])+"\n"
        graph += "}"
        print(graph)
    

    def connected_components(self):
        """Finds the connected components in the graph
        
        Returns
        -------
        components: list of lists
            each sub-list represents a connected component
        
        """
        self.connectedComponents = []
        treated_nodes = {}
        for i in self.nodes:
            treated_nodes[i] = False
        
        for i in self.nodes:
            if not treated_nodes[i]:
                temp = []
                self.DFS(i, treated_nodes, temp)
                self.connectedComponents.append(temp)
                
        return self.connectedComponents
    
    def DFS(self, nodeId, treated_nodes, component):
        component.append(nodeId)
        treated_nodes[nodeId] = True

        for j in self.nodes[nodeId].edges:
            if not treated_nodes[j]:
                self.DFS(j, treated_nodes, component)

    
    def connected_components_number(self):
        if not self.connectedComponents:
            self.connectedComponents = self.connected_components()
        return len(self.connectedComponents)
    
    def graph_to_matrix(G):
        matrix = []
        for i in G.nodes:
            ligne = [0 for i in range(len(G.nodes))]
            for j in G.nodes[i].edges:
                ligne[j] = G.nodes[i].edges[j]
            matrix.append(ligne)
        return matrix #np.array([m for m in matrix])
            
    def __str__(self):
        graph = "{"
        for node in self.nodes:
            graph += str(self.nodes[node])+"\n"
        graph += "}"
        return graph

In [3]:
class Node(object):
    """
    A class used to represent a node
    
    Attributes
    ----------
    id : int
        used to identify the node
    topic : str
        the topic of the article 
        represented by this node
    tokens: list of str
        list of the tokens of the article
    edges: list of int
        list of the nodes connected to this node
        
    Methods
    -------
    __str__()
        returns the id of the node with the edges 
    """
    def __init__(self, id, topic, tokens):
        self.id = id
        self.topic = topic
        self.tokens = list(tokens)
        self.edges = {}
        
    def __str__(self):  
        return "{" + str(self.id)+ ":" + str(self.edges) + "}\n"

In [4]:
def intersection(node1, node2):
    """Finds the number of tokens shared by two nodes
    
    Parameters
    ----------
    node1, node2 : node
    
    Returns
    -------
    int: number of tokens shared by the two nodes
    """
    return len(list(set(node1.tokens) & set(node2.tokens)))

In [5]:
# removing HTML tags from the articles
def remove_tags(text):
    """Remove HTML tags from a given text
    
    Parameters
    ----------
    text: str
        text to clean from html tags
        
    Returns
    -------
        : str
        clean text
    """
    TAG_RE = re.compile(r'<[^>]+>')
    return TAG_RE.sub('', text)

In [6]:
#Set of functions used to clean a text and transform it to a list of tokens 

def extract_tokens(text):
    """Extracts tokens from a given text
    
    Parameters
    ----------
    text: str
        text to be splited to tokens
    Returns
    -------
    res : list
        list of tokens in the text
    """
    res = []
    for sent in nltk.sent_tokenize(text):
        tmp_res = nltk.word_tokenize(sent)
        for token in tmp_res:
            res += re.split("[\./]", token)
    return res


def clean_tokens(tokens):
    """Removes punctuation marks from tokens
    Parameters
    ----------
    tokens: list
        list to be cleaned from punctuation marks
    
    Returns
    -------
    list:
        list cleaned from punctuation marks
    """
    return [token.lower() for token in tokens if token not in string.punctuation]


def remove_stop_words(tokens):
    """Removes stopwords from our tokens
    
    Parameters
    ----------
    tokens: list 
        list of tokens to be cleaned from stopwords
        
    Returns
    -------
        :list
        list cleaned from stopwords
    """
    GARBAGE = {"'s", "n't", '...', 'oh',"'m", "'re", "'", "''", "'ve", "'ll", "'d", "``" }
    STOP_WORDS = set(stopwords.words('english')).union(GARBAGE)
    return [token for token in tokens if token not in STOP_WORDS]

def text2tokens(text):
    """Used to combine three previous functions in one operation.
    Parameters
    ----------
    Text: str
        text to be cleaned
    Returns
    -------
    tokens: list
        list of tokens cleaned
    """
    text = remove_tags(text)
    tokens = extract_tokens(text)
    tokens = clean_tokens(tokens)
    tokens = remove_stop_words(tokens)
    return tokens

In [7]:
#loading dataset
df = pd.read_csv("dataset_business_technology_cybersecurity.zip")

In [8]:
df.sample(8)

,title,content,topic
114,Software engineering,<p><b>Software engineering</b> is the systemat...,technology
28,Company,"<p class=""mw-empty-elt"">\n</p>\n\n<p>A <b>comp...",business
285,Misuse case,<p><b>Misuse case</b> is a business process mo...,cybersecurity
111,Processor design,<p><b>Processor design</b> is the design engin...,technology
169,Automated attendant,"<p>In telephony, an <b>automated attendant</b>...",technology
46,Growth platforms,<p><br><b>Growth platforms</b> are specific in...,business
80,Outline of finance,<p>The following outline is provided as an ove...,business
56,Unemployment,"<p><b>Unemployment</b>, according to the OECD ...",business


In [9]:
df["tokens"] = df["content"].apply(lambda s: text2tokens(s))

In [10]:
#create a graph using articles as nodes
G = Graph()
for index, article in df.iterrows():
    G.add_node(Node(index, article["topic"], article["tokens"]))

In [11]:
#Adding edges between each two nodes if they share 100 tokens
G.add_edges(50)

In [12]:
#computing connected components
connected_components = G.connected_components()

In [15]:
for component in  connected_components:
    print(component)
    print("--------------------")

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 12, 13, 16, 17, 15, 18, 19, 20, 21, 22, 23, 25, 24, 29, 27, 28, 30, 31, 32, 33, 34, 35, 37, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 50, 48, 51, 52, 53, 54, 55, 56, 57, 58, 59, 64, 62, 65, 60, 67, 66, 68, 69, 72, 70, 71, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 63, 174, 91, 92, 93, 94, 95, 90, 98, 96, 97, 99, 100, 101, 103, 104, 105, 106, 107, 108, 102, 110, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 167, 170, 169, 172, 171, 179, 173, 176, 180, 175, 181, 178, 182, 183, 177, 184, 185, 186, 187, 188, 190, 189, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 229, 228